In [3]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.ensemble import IsolationForest
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
import time
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


import time

LABEL_INDEX = 14

In [4]:
full_dataset = pd.read_hdf("./events_anomalydetection_v2.features.h5")
# data_as_tuples = list(dataset.itertuples())
dataset = pd.concat([full_dataset[:10**5], full_dataset[-1*10**4:]])

In [5]:
# train_data = list(small_dataset.itertuples())
# [train_data[0]] + [train_data[-1]]

In [6]:
# print(len(train_data)
# Counter([i[-1] for i in train_data])
x = dataset.iloc[:, 0:LABEL_INDEX].values
y = dataset.iloc[:, LABEL_INDEX].values
print(str(x))
print(str(y))

[[-1467.23999023   611.50201416   511.10198975 ...    79.81510162
     21.01029968    16.75760078]
 [-1211.23999023   347.31500244   547.9630127  ...     8.0421896
      6.33509016     5.52536011]
 [-1229.61999512   649.85797119     8.08917046 ...    15.29290009
     13.94419956    10.01350021]
 ...
 [ -149.33000183  1781.45996094   -58.6908989  ...    79.12010193
     46.53730011    23.22730064]
 [ 1584.69995117  -731.15698242  -196.3480072  ...   366.18899536
    192.13900757    81.39820099]
 [-1229.59997559 -1009.89001465  1105.80004883 ...   238.96600342
    130.36999512    88.14409637]]
[0. 0. 0. ... 1. 1. 1.]


In [7]:

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
# print(len(x_train))
# print(len(x_test))
# print(Counter(y_train))
# print(Counter(y_test))

In [8]:

sc = StandardScaler()
scaled_x_train = sc.fit_transform(x_train)
scaled_x_test = sc.transform(x_test)
print(x_train[0:2])
print(scaled_x_train[0:2])

[[  912.41497803   904.66900635  -478.95999146   214.54899597
    159.1190033     82.74919891    62.88499832  -876.54302979
   -880.28399658  -897.42797852   140.80700684    44.10860062
     31.31110001    18.70639992]
 [ 1113.98999023 -1019.22998047   935.20898438   149.28399658
     64.17189789    41.30039978    33.39789963   587.58099365
   1086.42004395   -84.81600189   663.21899414   568.44299316
    224.03999329   109.90899658]]
[[ 0.90801941  0.90309745 -0.4097019  -0.02183458  0.1731204   0.55435299
   0.85581705 -0.98818547 -0.99478724 -0.77247743 -0.44404696 -0.65827286
  -0.47034895 -0.61041917]
 [ 1.10899263 -1.01808479  0.82055994 -0.41929821 -0.51579258 -0.28480139
  -0.1271039   0.66414832  1.22845482 -0.07298144  2.96275329  3.40840354
   3.64227748  2.60611955]]


In [9]:
# took about 20 seconds
regressor = RandomForestRegressor(n_estimators=20, random_state=0)
regressor.fit(scaled_x_train, y_train)
y_pred = regressor.predict(scaled_x_test)

In [10]:
print(len(y_pred))
pred_signal = [i for i in range(len(y_pred)) if y_pred[i] == 1] 
print(len(pred_signal))
print(len([i for i in pred_signal if y_test[i] == 1]))
# print(len([i for i in range(len(y_pred)) if y_pred[i] == y_test[i]]))

22000
626
619


In [11]:
# complete regressor
x = full_dataset.iloc[:, 0:LABEL_INDEX].values
y = full_dataset.iloc[:, LABEL_INDEX].values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

sc = StandardScaler()
scaled_x_train = sc.fit_transform(x_train)
scaled_x_test = sc.transform(x_test)

regressor = RandomForestRegressor(n_estimators=20, random_state=0)
print(f"start {time.time()}")
regressor.fit(scaled_x_train, y_train)
print(f"fit {time.time()}")
y_pred = regressor.predict(scaled_x_test)
print(f"pred {time.time()}")


pred_signal = [i for i in range(len(y_pred)) if y_pred[i] == 1]
real_signal = [i for i in pred_signal if y_test[i] == 1]
print(len(y_pred))
print(len(pred_signal))
print(len(real_signal))
print(len(real_signal) / len(pred_signal))
"""
for full_dataset:
start 1624715840.3091583
fit 1624716328.8221645
pred 1624716329.6628819
220000
7055
6957
0.9861091424521616
"""

start 1631970461.863416
fit 1631971456.574479
pred 1631971458.305404
220000
7055
6957
0.9861091424521616


'\nfor full_dataset:\nstart 1624715840.3091583\nfit 1624716328.8221645\npred 1624716329.6628819\n220000\n7055\n6957\n0.9861091424521616\n'

In [12]:

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))


Mean Absolute Error: 0.04905454545454545
Mean Squared Error: 0.02391215909090909
Root Mean Squared Error: 0.154635568647414


In [13]:

# took about 20 seconds
classifier = RandomForestClassifier(n_estimators=20, random_state=0)
print(f"start {time.time()}")
classifier.fit(scaled_x_train, y_train)
print(f"fitted {time.time()}")
cls_y_pred = classifier.predict(scaled_x_test)
print(f"predicted {time.time()}")

print(len(cls_y_pred))
cls_pred_signal = [i for i in range(len(cls_y_pred)) if cls_y_pred[i] == 1] 
print(len(cls_pred_signal))
print(len([i for i in cls_pred_signal if y_test[i] == 1]))

start 1631971458.527349
fitted 1631971761.809546
predicted 1631971763.600317
220000
16349
15001


In [14]:
## Isolation Forest - 13% with not labled data
## https://scikit-learn.org/stable/modules/outlier_detection.html

# background_train_dataset = full_dataset[:10**6 - 10 ** 5].iloc[:, 0:LABEL_INDEX].values
# background_test_dataset = full_dataset[10**6 - 10 ** 5:].iloc[:, 0:LABEL_INDEX].values # should be half signal
# train_set = background_train_dataset
# test_set = background_test_dataset
# is_signal = lambda i: i > 10**5


# x = full_dataset.iloc[:, 0:LABEL_INDEX].values
# y = full_dataset.iloc[:, LABEL_INDEX].values
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
# train_set = x_train
# test_set = x_test
# is_signal = lambda i: y_test[i] == 1

train_set = test_set = full_dataset.iloc[:, 0:LABEL_INDEX].values
is_signal = lambda i: i > 10**6

# x = full_dataset.iloc[:, 0:LABEL_INDEX].values
# y = full_dataset.iloc[:, LABEL_INDEX].values
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
# train_set = test_set = x_train
# is_signal = lambda i: y_train[i] == 1

# sc = StandardScaler()
# scaled_train = sc.fit_transform(scaled_train)
# scaled_test = sc.transform(scaled_test)

iso_forest = IsolationForest(n_estimators=100, random_state=0, contamination=0.09)
print(f"start {time.time()}")
iso_forest.fit(train_set)
print(f"fitted {time.time()}")
iso_y_pred = iso_forest.predict(test_set)
print(f"predicted {time.time()}")

iso_pred_signal = [i for i in range(len(iso_y_pred)) if iso_y_pred[i] != 1]
count_real_signal = len([i for i in iso_pred_signal if is_signal(i)])
print(f"total tested {len(iso_y_pred)}")
print(f"found signal {len(iso_pred_signal)}")
print(f"real  signal {count_real_signal}")
print(f"ratio signal {count_real_signal / len(iso_pred_signal)}")


start 1631971763.748289
fitted 1631971835.55857
predicted 1631971901.029619
total tested 1100000
found signal 99000
real  signal 13307
ratio signal 0.1344141414141414


In [15]:
import matplotlib.pyplot as plt
print(train_set[1])

[-1211.23999023   347.31500244   547.9630127    389.53201294
   191.80400085    99.56279755    70.87220001   619.34100342
   -62.1772995  -1944.04003906    22.99920082     8.0421896
     6.33509016     5.52536011]


In [ ]:
## local outline factor - 6% (less than starting ration) with not labled data
import os
from sklearn.neighbors import LocalOutlierFactor

train_set = test_set = full_dataset.iloc[:, 0:LABEL_INDEX].values
is_signal = lambda i: i > 10**6

# perdictor = LocalOutlierFactor(n_neighbors=35, contamination=0.09)
perdictor = LocalOutlierFactor(n_neighbors=35, contamination=0.09, n_jobs=10)
print(f"start {time.time()}")
y_pred = perdictor.fit_predict(train_set)
print(f"pred {time.time()}")

pred_signal = [i for i in range(len(y_pred)) if y_pred[i] != 1]
count_real_signal = len([i for i in pred_signal if is_signal(i)])
real_signal = [i for i in range(len(train_set)) if is_signal(i)]

print(f"total tested {len(y_pred)}")
print(f"found signal {len(pred_signal)}")
print(f"real  signal {count_real_signal}")
print(f"ratio signal {count_real_signal / len(pred_signal)}")
print(f"recall: {count_real_signal/len(real_signal)}")
os.system("printf '\a'")

In [17]:
## local outline factor +  Isolation Forest 18% with not labled data

combined_signal = list(set(iso_pred_signal) & set(pred_signal))
count_real_signal = len([i for i in pred_signal if is_signal(i)])
# print(f"total tested {len(y_pred)}")
print(f"found signal {len(combined_signal)}")
print(f"real  signal {count_real_signal}")
print(f"ratio signal {count_real_signal / len(combined_signal)}")
os.system("printf '\a'")

found signal 35964
real  signal 6780
ratio signal 0.18852185518852185


0

In [18]:
### SVM - slow, 13% when training on background and then checking on mix
# from here https://scikit-learn.org/stable/modules/outlier_detection.html
# background_train_dataset = full_dataset[:10**6 - 10 ** 5].iloc[:, 0:LABEL_INDEX].values
# test_dataset = full_dataset[10 ** 6 - 10 ** 5:10 ** 6 + 10 ** 4].iloc[:, 0:LABEL_INDEX].values
# train_dataset = dataset.iloc[:, 0:LABEL_INDEX]

## train on little data
x = full_dataset.iloc[:, 0:LABEL_INDEX].values
y = full_dataset.iloc[:, LABEL_INDEX].values
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.99, random_state=0)
train_dataset = test_dataset = x_train
is_signal = lambda i: y_train[i]


## train on background only 
# N = 4
# train_dataset = full_dataset[:10 ** N].iloc[:, 0:LABEL_INDEX].values
# test_dataset = full_dataset[10 ** 6 - 10 ** N : 10 ** 6 + 10 ** (N - 1)].iloc[:, 0:LABEL_INDEX].values
# is_signal = lambda i: i > 10 ** N

# dataset = pd.concat([full_dataset[:10**4], full_dataset[-1*10**3:]])
# is_signal = lambda i: i > 10**5

print(len(train_dataset))
print(len(test_dataset))


from sklearn import svm
onc_class_svm = svm.OneClassSVM(nu=0.1, kernel="rbf")
print(f"start {time.time()}")
# onc_class_svm.fit(train_dataset)
# print(f"fit {time.time()}")
# y_pred = onc_class_svm.predict(test_dataset)
y_pred = onc_class_svm.fit_predict(train_dataset)
print(f"pred {time.time()}")

pred_signal = [i for i in range(len(y_pred)) if y_pred[i] != 1]
count_real_signal = len([i for i in pred_signal if is_signal(i)])
print(f"total tested {len(y_pred)}")
print(f"found signal {len(pred_signal)}")
print(f"real  signal {count_real_signal}")
print(f"ratio signal {count_real_signal / len(pred_signal)}")

11000
11000
start 1631972295.204567
pred 1631972301.255411
total tested 11000
found signal 1101
real  signal 143
ratio signal 0.1298819255222525


In [22]:
# local outline factor
from sklearn.neighbors import LocalOutlierFactor
full_dataset = pd.read_hdf("./events_anomalydetection_v2.features.h5")

# x = full_dataset.iloc[:, 0:LABEL_INDEX].values
# y = full_dataset.iloc[:, LABEL_INDEX].values
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.9, random_state=0)
# train_dataset = test_dataset = x_train
# is_signal = lambda i: y_train[i]

## train on background only 
N = 5
train_dataset = full_dataset[:10 ** N].iloc[:, 0:LABEL_INDEX].values
test_dataset = full_dataset[10 ** 6 - 10 ** N : 10 ** 6 + 10 ** (N - 1)].iloc[:, 0:LABEL_INDEX].values
is_signal = lambda i: i > 10 ** N


# perdictor = LocalOutlierFactor(n_neighbors=35, contamination=0.09)
perdictor = LocalOutlierFactor(n_neighbors=200, contamination=0.09, novelty=True, n_jobs=10)
print(f"start {time.time()}")
perdictor.fit(train_dataset)
print(f"fit {time.time()}")
y_pred = perdictor.predict(test_dataset)
# y_pred = perdictor.fit_predict(train_dataset)
print(f"pred {time.time()}")

pred_signal = [i for i in range(len(y_pred)) if y_pred[i] != 1]
count_real_signal = len([i for i in pred_signal if is_signal(i)])
real_signal = [i for i in range(len(test_dataset)) if is_signal(i)]

print(f"total tested {len(y_pred)}")
print(f"found signal {len(pred_signal)}")
print(f"real  signal {count_real_signal}")
print(f"ratio signal {count_real_signal / len(pred_signal)}")
print(f"recall: {count_real_signal/len(real_signal)}")
"""
for n = 5
start 1624726190.1007173
fit 1624726201.004385
pred 1624726213.5636365
total tested 110000
found signal 12396
real  signal 3653
ratio signal 0.2946918360761536
"""

start 1632840927.280201
fit 1632841022.25182
pred 1632841145.2981088
total tested 110000
found signal 12396
real  signal 3653
ratio signal 0.2946918360761536
recall: 0.3653365336533653


'\nfor n = 5\nstart 1624726190.1007173\nfit 1624726201.004385\npred 1624726213.5636365\ntotal tested 110000\nfound signal 12396\nreal  signal 3653\nratio signal 0.2946918360761536\n'

In [20]:
# https://github.com/lmcinnes/umap
import umap
import umap.plot

x = test_dataset
y = y_pred
mapper = umap.UMAP().fit(x)
umap.plot.points(mapper, labels=y )

# x = full_dataset[10**6-10000:10**6+1000].iloc[:, 0:LABEL_INDEX].values
# y = full_dataset[10**6-10000:10**6+1000].iloc[:, LABEL_INDEX].values
# mapper = umap.UMAP().fit(x)
# umap.plot.points(mapper, labels=y )

# digits = load_digits()
# mapper = umap.UMAP().fit(digits.data)
# umap.plot.points(mapper, labels=digits.target)

ModuleNotFoundError: No module named 'umap'